# Test run_analysis Handler Locally

This notebook tests the run_analysis handler without S3 uploads.
Files will be saved to `notebooks/data/test-results/` instead.

In [1]:
import sys
sys.path.insert(0, '..')

import os
import json
from datetime import datetime
import pandas as pd
import torch

from shared.brain_data import load_brain_data
from shared.feature_data import load_feature_data
from shared.analysis import doBrainAndFeaturePrediction

## Configuration

Set up test parameters (same as what the Lambda handler would receive)

In [2]:
# Test configuration (simulates Lambda event body)
config = {
    'brain_subject': 1,
    'year': '2020',
    'group_name': 'dopaminemachine',
    'num_voxels': 500,
    'zscore_braindata': False,
    'testIndividualFeatures': True,  # Set to False for faster testing
    'analysis_id': 'test-local-001'
}

# Local output directory (instead of S3)
output_dir = 'data/test-results'
os.makedirs(output_dir, exist_ok=True)

print("Test Configuration:")
print(json.dumps(config, indent=2))
print(f"\nOutput directory: {output_dir}")

Test Configuration:
{
  "brain_subject": 1,
  "year": "2020",
  "group_name": "dopaminemachine",
  "num_voxels": 500,
  "zscore_braindata": false,
  "testIndividualFeatures": true,
  "analysis_id": "test-local-001"
}

Output directory: data/test-results


## Load Data

Test loading brain and feature data

In [3]:
print(f"Loading brain data for subject {config['brain_subject']}...")
brain_data = load_brain_data(config['brain_subject'])

print(f"Loading feature data for {config['year']}/{config['group_name']}...")
feature_data = load_feature_data(year=config['year'], group_name=config['group_name'])

print(f"\nBrain data shape: {brain_data['D'].shape}")
print(f"Feature data shape: {feature_data['R'].shape}")
print(f"Number of features: {len(feature_data['featureNames'])}")
print(f"Feature names: {feature_data['featureNames']}")

Loading brain data for subject 1...
Using cached brain data: /tmp/data-science-P1_converted.mat
Loading feature data for 2020/dopaminemachine...
Using cached feature data: /tmp/dopaminemachine_Ratings.csv
Found 60 items, 16 features, 3 raters

Brain data shape: (60, 21764, 6)
Feature data shape: (60, 16)
Number of features: 16
Feature names: ['animaltype' 'clothing' 'container' 'edible' 'electronics' 'hardness'
 'humanbodypart' 'living' 'moldable' 'moving' 'shape' 'size' 'taste'
 'tool' 'transparency' 'vehicle']


## Run Analysis

Execute the full 1770-iteration analysis with progress tracking

In [4]:
from tqdm import tqdm

# Progress tracking
pbar = tqdm(total=1770, desc="Analysis progress")
start_time = datetime.utcnow()

def progress_callback(current, total):
    pbar.update(1)
    if current % 100 == 0 or current == total:
        elapsed = (datetime.utcnow() - start_time).total_seconds()
        rate = current / elapsed if elapsed > 0 else 0
        pbar.set_postfix({
            'iter/s': f'{rate:.1f}',
            'ETA': f'{((total - current) / rate / 60):.1f}m' if rate > 0 else 'N/A'
        })

# Run analysis
print("Starting analysis (1770 iterations)...\n")
results = doBrainAndFeaturePrediction(
    brain_data=brain_data,
    feature_data=feature_data,
    num_voxels=config['num_voxels'],
    zscore_braindata=config['zscore_braindata'],
    shuffle_features=False,
    testIndividualFeatures=config['testIndividualFeatures'],
    progress_callback=progress_callback
)

pbar.close()

end_time = datetime.utcnow()
elapsed_time = (end_time - start_time).total_seconds()

print(f"\nAnalysis complete!")
print(f"Elapsed time: {elapsed_time:.1f}s ({elapsed_time/60:.2f} min)")
print(f"Keys in results: {results.keys()}")

Analysis progress:   0%|                             | 1/1770 [00:00<00:05, 328.71it/s, iter/s=0.0, ETA=N/A]

Starting analysis (1770 iterations)...

ANALYZING SUBJECT NUMBER: 1


Analysis progress: 100%|██████████████████████████| 1770/1770 [06:37<00:00,  4.45it/s, iter/s=4.4, ETA=0.3m]


Analysis complete!
Elapsed time: 397.4s (6.62 min)
Keys in results: dict_keys(['results', 'results_by_feature', 'all_betas'])


## Convert to DataFrames

Test converting results to pandas DataFrames (same as Lambda handler)

In [10]:
print("Converting results to DataFrames...")
results_df = pd.DataFrame(results['results'])

results_by_feature_df = None
if results['results_by_feature']:
    results_by_feature_df = pd.DataFrame(results['results_by_feature'])

print(f"\nResults DataFrame shape: {results_df.shape}")
print(f"Columns: {list(results_df.columns)}")

if results_by_feature_df is not None:
    print(f"\nResults by feature DataFrame shape: {results_by_feature_df.shape}")
    print(f"Columns: {list(results_by_feature_df.columns)}")

# Preview
print("\nFirst few rows:")
results_df.head()

Converting results to DataFrames...

Results DataFrame shape: (14160, 18)
Columns: ['brain_subject', 'item1_idx', 'item2_idx', 'item1_name', 'item2_name', 'item1_cat', 'item2_cat', 'itemPair', 'same_category', 'r2_score', 'task', 'method', 'scoring', 'dist11', 'dist22', 'dist12', 'dist21', 'correct']

Results by feature DataFrame shape: (56640, 20)
Columns: ['feat_num', 'feat_name', 'brain_subject', 'item1_idx', 'item2_idx', 'item1_name', 'item2_name', 'item1_cat', 'item2_cat', 'itemPair', 'same_category', 'r2_score', 'task', 'method', 'scoring', 'dist11', 'dist22', 'dist12', 'dist21', 'correct']

First few rows:


,brain_subject,item1_idx,item2_idx,item1_name,item2_name,item1_cat,item2_cat,itemPair,same_category,r2_score,task,method,scoring,dist11,dist22,dist12,dist21,correct
0,1,0,1,bear,cat,animal,animal,"(0, 1)",1,0.544311,brain_prediction,encoding_model,individual,0.430449,0.517984,0.735480,0.500119,0.5
1,1,0,1,bear,cat,animal,animal,"(0, 1)",1,0.544311,brain_prediction,encoding_model,combo,0.430449,0.517984,0.735480,0.500119,1.0
2,1,0,1,bear,cat,animal,animal,"(0, 1)",1,0.544311,mind_reading,encoding_model,individual,0.420105,0.707555,0.455400,1.036087,1.0
3,1,0,1,bear,cat,animal,animal,"(0, 1)",1,0.544311,mind_reading,encoding_model,combo,0.420105,0.707555,0.455400,1.036087,1.0
4,1,0,1,bear,cat,animal,animal,"(0, 1)",1,0.544311,brain_prediction,botastic_templates,individual,0.412989,0.500988,0.616381,0.386240,0.5


## Save Files Locally

Save all files to local directory (instead of S3)

In [6]:
print(f"Saving files to {output_dir}/...\n")

# Save CSVs
results_csv_path = f'{output_dir}/results.csv'
results_df.to_csv(results_csv_path, index=False)
print(f"✓ Saved results.csv ({os.path.getsize(results_csv_path) / 1024 / 1024:.2f} MB)")

if results_by_feature_df is not None:
    results_by_feature_csv_path = f'{output_dir}/results_by_feature.csv'
    results_by_feature_df.to_csv(results_by_feature_csv_path, index=False)
    print(f"✓ Saved results_by_feature.csv ({os.path.getsize(results_by_feature_csv_path) / 1024 / 1024:.2f} MB)")

# Save betas
all_betas_path = f'{output_dir}/all_betas.pth'
torch.save(results['all_betas'], all_betas_path)
print(f"✓ Saved all_betas.pth ({os.path.getsize(all_betas_path) / 1024 / 1024:.2f} MB)")

# Get actual number of iterations from results
num_iterations = len(results['all_betas'])

# Save config
config_data = {
    **config,
    'timestamp': start_time.isoformat(),
    'elapsed_time': elapsed_time,
    'num_iterations': num_iterations,  # Use actual count instead of hardcoded 1770
    'num_features': len(feature_data['featureNames']),
    'feature_names': feature_data['featureNames'].tolist()
}
config_path = f'{output_dir}/config.json'
with open(config_path, 'w') as f:
    json.dump(config_data, f, indent=2)
print(f"✓ Saved config.json ({os.path.getsize(config_path) / 1024:.2f} KB)")

print(f"\nAll files saved to: {os.path.abspath(output_dir)}/")

Saving files to data/test-results/...

✓ Saved results.csv (2.55 MB)
✓ Saved results_by_feature.csv (10.83 MB)
✓ Saved all_betas.pth (168.32 MB)
✓ Saved config.json (0.58 KB)

All files saved to: /workspaces/neuro_science_fiction/backend/mitchell/notebooks/data/test-results/


## Compute Summary Statistics

Test the summary statistics function (same as Lambda handler)

In [11]:
# Import the summary function from the handler
from handlers.run_analysis import compute_summary_statistics

# Get actual iteration count from results
num_iterations = len(results['all_betas'])

summary = compute_summary_statistics(results_df, elapsed_time, num_iterations)

print("Summary Statistics:")
print(json.dumps(summary, indent=2))

Summary Statistics:
{
  "num_iterations": 1770,
  "elapsed_time": 397.35,
  "elapsed_time_minutes": 6.62,
  "brain_prediction_encoding_model_individual": 0.8576,
  "brain_prediction_encoding_model_combo": 0.9316,
  "brain_prediction_botastic_templates_individual": 0.8768,
  "brain_prediction_botastic_templates_combo": 0.9486,
  "mind_reading_encoding_model_individual": 0.8189,
  "mind_reading_encoding_model_combo": 0.9107,
  "mind_reading_botastic_templates_individual": 0.8749,
  "mind_reading_botastic_templates_combo": 0.9395,
  "same_category_accuracy": 0.6667,
  "different_category_accuracy": 0.9509,
  "num_same_category": 120,
  "num_different_category": 1650,
  "mean_r2_score": 0.5495
}


## Verify Results

Quick sanity checks on the results

In [8]:
print("Verification Checks:\n")

# Check number of rows
expected_rows = 1770 * 8  # 1770 pairs × 8 combinations (2 tasks × 2 methods × 2 scorings)
actual_rows = len(results_df)
print(f"✓ Expected {expected_rows} rows, got {actual_rows}" if actual_rows == expected_rows else f"✗ Expected {expected_rows} rows, got {actual_rows}")

# Check accuracy ranges
for key, value in summary.items():
    if 'accuracy' in key and value is not None:
        if 0 <= value <= 1:
            print(f"✓ {key}: {value:.4f} (valid range)")
        else:
            print(f"✗ {key}: {value:.4f} (out of range!)")

# Check betas
print(f"\n✓ Number of beta matrices: {len(results['all_betas'])}")
if len(results['all_betas']) > 0 and results['all_betas'][0] is not None:
    print(f"✓ Beta matrix shape: {results['all_betas'][0].shape}")

print("\n🎉 All checks passed!")

Verification Checks:

✓ Expected 14160 rows, got 14160
✓ same_category_accuracy: 0.6667 (valid range)
✓ different_category_accuracy: 0.9509 (valid range)

✓ Number of beta matrices: 1770
✓ Beta matrix shape: (500, 16)

🎉 All checks passed!


## Explore Results

Quick analysis of the results

In [9]:
# Accuracies by task/method/scoring
pivot = results_df.groupby(['task', 'method', 'scoring'])['correct'].mean().round(4)
print("Accuracies by task/method/scoring:")
print(pivot)

print("\n" + "="*60)

# Same vs different category
brain_pred_subset = results_df[
    (results_df['task'] == 'brain_prediction') &
    (results_df['method'] == 'encoding_model') &
    (results_df['scoring'] == 'combo')
]
same_cat_acc = brain_pred_subset[brain_pred_subset['same_category'] == 1]['correct'].mean()
diff_cat_acc = brain_pred_subset[brain_pred_subset['same_category'] == 0]['correct'].mean()

print(f"Same category accuracy: {same_cat_acc:.4f}")
print(f"Different category accuracy: {diff_cat_acc:.4f}")

Accuracies by task/method/scoring:
task              method              scoring   
brain_prediction  botastic_templates  combo         0.9486
                                      individual    0.8768
                  encoding_model      combo         0.9316
                                      individual    0.8576
mind_reading      botastic_templates  combo         0.9395
                                      individual    0.8749
                  encoding_model      combo         0.9107
                                      individual    0.8189
Name: correct, dtype: float64

Same category accuracy: 0.6667
Different category accuracy: 0.9509
